In [1]:
import pickle
import numpy as np
import pandas as pd
import datetime

In [2]:
import torch
def compute_loss(sub_df,y):
    pred = torch.Tensor(np.log(sub_df.loc[:,['A','B','NEITHER']].values))
    label = torch.LongTensor(list(y))
    loss = torch.nn.NLLLoss()
    loss_value = loss(pred,label).item()
    return loss_value

In [3]:
f = open("./temp_result/biDAF_result", "rb")
train1 = pickle.load(f)
test1 = pickle.load(f)
f.close()

In [4]:
f = open("./temp_result/mlp_result", "rb")
train2 = pickle.load(f)
test2 = pickle.load(f)
f.close()

In [5]:
f = open("./temp_result/svm_result", "rb")
train3 = pickle.load(f)
test3 = pickle.load(f)
f.close()

In [6]:
f = open("./temp_result/lr_result", "rb")
train4 = pickle.load(f)
test4 = pickle.load(f)
f.close()

In [7]:
f = open("./temp_result/label", "rb")
y_train = pickle.load(f)
y_test = pickle.load(f)
f.close()

In [8]:
import pickle
f = open( "./temp_result/base_model_data", "rb" )
X_train = pickle.load(f)
y_train = pickle.load(f)
X_valid = pickle.load(f)
y_valid = pickle.load(f)
X_test = pickle.load(f)
y_test = pickle.load(f)
f.close()

In [18]:
#X_train = np.concatenate([train1,train2,train3,train4,X_train],axis = 1)
#X_test = np.concatenate([test1,test2,test3,test4,X_test],axis = 1)
X_train = np.concatenate([train1,train2,train3,X_train],axis = 1)
X_test = np.concatenate([test1,test2,test3,X_test],axis = 1)

### LR

In [10]:
from sklearn.linear_model import LogisticRegression

lr_stack = LogisticRegression(penalty = "l1",random_state=0,solver = "saga",multi_class='multinomial',class_weight='balanced', C= 0.035).fit(X_train, y_train)

/home/bao/anaconda3/envs/EPFL/lib/python3.6/site-packages/sklearn/linear_model/sag.py:334: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


In [11]:
pred_lr_stack = lr_stack.predict_proba(X_test)

In [12]:
#pred_lr = process_prediction(pred_lr)
sub_df = pd.read_csv("./test_and_submit/sample_submission_stage_1.csv")
sub_df.loc[:, ['A','B','NEITHER']] = pred_lr_stack


sub_df.to_csv("./test_and_submit/submission+model+lr_stack@"+str(datetime.datetime.now())+".csv", index=False)

sub_df.head(7)

,ID,A,B,NEITHER
0,development-1,0.950964,0.030939,0.018097
1,development-2,0.964662,0.021400,0.013937
2,development-3,0.036537,0.915843,0.047620
3,development-4,0.068720,0.755637,0.175643
4,development-5,0.014912,0.972950,0.012138
5,development-6,0.971878,0.016850,0.011272
6,development-7,0.781265,0.111409,0.107326


In [13]:
compute_loss(sub_df,y_test)

0.46631428599357605

### SVM

In [14]:
from sklearn.svm import SVC
svm_stack = SVC(C = 0.5,verbose=True,probability = True,gamma = "auto",class_weight='balanced').fit(X_train, y_train)

[LibSVM]

In [15]:
pred_svm_stack = svm_stack.predict_proba(X_test)

In [16]:
#pred_lr = process_prediction(pred_lr)
sub_df = pd.read_csv("./test_and_submit/sample_submission_stage_1.csv")
sub_df.loc[:, ['A','B','NEITHER']] = pred_svm_stack


sub_df.to_csv("./test_and_submit/submission+model+svm_stack@"+str(datetime.datetime.now())+".csv", index=False)

sub_df.head(7)

,ID,A,B,NEITHER
0,development-1,0.836139,0.131227,0.032633
1,development-2,0.986470,0.010850,0.002680
2,development-3,0.009299,0.987728,0.002972
3,development-4,0.023183,0.861042,0.115775
4,development-5,0.006385,0.988271,0.005344
5,development-6,0.998422,0.001439,0.000139
6,development-7,0.861572,0.031631,0.106796


In [17]:
compute_loss(sub_df,y_test)

0.5611711144447327